In [38]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse import csc_matrix,diags,lil_matrix,vstack
from scipy.sparse.linalg import spsolve,inv
from scipy.linalg import det
from scipy.spatial import Delaunay
from IPython.display import display, HTML,IFrame
import scipy.sparse as sp

interactive = True
use_cholesky = False
filename = 'woody-hi'
html=False
triangulation_methods=4 # 0 for scipy, 1 for triangle (removing unnecessary triangular), 2 for mean value, 3 for triangle with random sample points

In [39]:
v, f = igl.read_triangle_mesh(f'data/{filename}.off')
cage_verts=np.load("data/woody-hi.cage.npy")
v -= v.min(axis=0)
v /= v.max()

# Triangulation


In [48]:
seg=[]
for i in range(cage_verts.shape[0]):
        seg.append([i,(i+1)%cage_verts.shape[0]])
if triangulation_methods==0:
    tri_verts=np.vstack((cage_verts,v))[:,:2]
    temp=Delaunay(tri_verts)
    tri_verts,tri_faces=temp.points,temp.simplices
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})
elif triangulation_methods==1:
    import triangle
    tri_verts=np.vstack((cage_verts,v))[:,:2]
    t = triangle.triangulate({'vertices': tri_verts,'segments':seg},'p')
    tri_verts=np.array(t['vertices'])
    tri_faces=np.array(t['triangles'])
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})
elif triangulation_methods==2:    
    import triangle
    t = triangle.triangulate({'vertices': cage_verts[:,:2],'segments':seg},'p')
    tri_verts=np.array(t['vertices'])
    tri_faces=np.array(t['triangles'])
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    print(tri_verts.shape)
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})
elif triangulation_methods==3:    
    import triangle
    t = triangle.triangulate({'vertices': cage_verts[:,:2],'segments':seg},'pa0.01')
    tri_verts=np.array(t['vertices'])
    tri_faces=np.array(t['triangles'])
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    print(tri_verts.shape)
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})

(21, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4299999…

In [49]:
handle_vertex_positions = cage_verts.copy()
pos_f_saver = np.zeros((cage_verts.shape[0], 3))
def pos_f(s,x,y,z):
    v_slice = cage_verts[s] + np.array([x,y,z])
    handle_vertex_positions[s] = v_slice
    pos_f_saver[s - 1] = [x,y,z]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    global lines_obj
    p.update_object(vertices = v_deformed,colors=np.array(weights[:,s]))
    p.remove_object(lines_obj)
    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))
    t1 = time.time()
    print('FPS', 1/(t1 - t0))

In [50]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(cage_verts.shape[0]))
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    #rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    #widgets_dict.update(rotate_widget)
    return widgets_dict

# Find the triangle in which a vertex is.

In [51]:
def mybarycentric(p, a, b, c): # solve barycentric weight of a point
    v0=b-a
    v1=c-a
    v2=p-a
    d00=np.dot(v0,v0)
    d01=np.dot(v0,v1)
    d11=np.dot(v1,v1)
    d20=np.dot(v2,v0)
    d21=np.dot(v2,v1)
    denom=d00*d11-d01*d01
    v=(d11*d20-d01*d21)/denom
    w=(d00*d21-d01*d20)/denom
    u=1.0-v-w
    return np.array([u,v,w])

def find_barycentric_triangle(obj_verts,tri_verts,tri_faces):
    index=np.zeros(obj_verts.shape[0],dtype="int")
    weight=np.zeros((obj_verts.shape[0],3))
    for i in range(obj_verts.shape[0]):
        for j in range(tri_faces.shape[0]):
            res=mybarycentric(obj_verts[i][:2],tri_verts[tri_faces[j,0]][:2],tri_verts[tri_faces[j,1]][:2],tri_verts[tri_faces[j,2]][:2])
            if res[0]>=0 and res[1]>=0 and res[2]>=0:
                index[i]=j
                weight[i]=res
                break
    return index,weight
if triangulation_methods>=2:
    index,barycentric_weight=find_barycentric_triangle(v,tri_verts,tri_faces)


# Find new free vertices and constraint vertices.

If the triangulation add some points on the boundary(cage), then they have to be contraints.

In [52]:
def find_new_free_vertices(tri_verts,tri_faces,cage_verts,segment): #in random sample point method, there might be new points on boundary who cannot be free
    index=[] # free index in tri_verts
    nonfreeinedge=[] # 
    nonfreeedgeweight=[]
    for i in range(tri_verts.shape[0]):
        flag=False
        for j in segment:
            if np.allclose(tri_verts[i],cage_verts[j[0]]):
                flag=True
                nonfreeinedge.append(j)
                nonfreeedgeweight.append([1,0])
                break
            if np.allclose(tri_verts[i],cage_verts[j[1]]):
                flag=True
                nonfreeinedge.append(j)
                nonfreeedgeweight.append([0,1])
                break
            e1=cage_verts[j[0]]-cage_verts[j[1]]
            e2=tri_verts[i]-cage_verts[j[1]]
            norm1=np.linalg.norm(e1)
            norm2=np.linalg.norm(e2)
            cos_value=np.dot(e1,e2)/(norm1*norm2)
            length=np.dot(e1,e2)/(norm1)
            if np.allclose(cos_value,1) and length>0 and length<norm1:
                nonfreeinedge.append(j)
                nonfreeedgeweight.append([length,1-length])
                flag=True
                break
        if flag==False:
            index.append(i)
    return index,nonfreeinedge,nonfreeedgeweight

if triangulation_methods>=2:
    free_index_tri,nonfreeinedge,nonfreeedgeweight=find_new_free_vertices(tri_verts,tri_faces,cage_verts,seg)
else:
    free_index_tri=[i for i in range(cage_verts.shape[0],tri_verts.shape[0])]
    nonfreeinedge=seg
    nonfreeedgeweight=np.ones((cage_verts.shape[0],2))
    nonfreeedgeweight[:,1]=0

# Solve every interior vertex's barycentric position. 
If the interior vertex is already on the trangular corner of cage, then no barycentric is needed to approximate it.

In [53]:
def valuebybarycentric(weights,value):
    return weights@value

def cagesolver_2d(tri_verts,tri_faces,cage_verts,obj_verts,free_indices):
    inverse_mask=np.ones(tri_verts.shape[0],dtype=bool)
    inverse_mask[free_indices]=False
    weights_res=np.zeros((obj_verts.shape[0],cage_verts.shape[0]))
    mass_mat=igl.massmatrix(tri_verts,tri_faces)
    M=mass_mat
    cot_mat=igl.cotmatrix(tri_verts,tri_faces)
    L=inv(M)@cot_mat
    Lff=L[free_indices,:][:,free_indices]
    Lfc=L[free_indices,:][:,inverse_mask]
    xc=np.zeros((inverse_mask.sum(),cage_verts.shape[0]))
    xc[:cage_verts.shape[0],:]=np.eye(cage_verts.shape[0])
    for i in range(cage_verts.shape[0],inverse_mask.sum()):
        xc[i,:]=valuebybarycentric(nonfreeedgeweight[i],xc[nonfreeinedge[i]])
    free_result=spsolve(Lff,-Lfc@xc) # result weight is for any cage points not on boundary
    if triangulation_methods<=1:
        return free_result
        
    result2=np.zeros((tri_verts.shape[0],cage_verts.shape[0]))
    result2[inverse_mask]=xc
    result2[free_indices]=free_result
    

    for i in range(obj_verts.shape[0]):
        i_in_face=index[i]
        weights_res[i]=valuebybarycentric(barycentric_weight[i],result2[tri_faces[i_in_face]])
    
    return weights_res
    
def cage_deformer(tri_verts):
    res=valuebybarycentric(weights,tri_verts)
    return res
pos_f.deformer=cage_deformer

In [54]:
weights=cagesolver_2d(tri_verts,tri_faces,cage_verts,v,free_index_tri)
#print(weights.sum(axis=1))

In [55]:
## Widget UI
if interactive==True:
    p = mp.plot(v, f, c=weights[:,0])

    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))

    iw.interact(pos_f, **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…